# Multiple bar chart showing the EU electricity production by source in 2023

## Get the data

Load the data into a [Pandas](https://pandas.pydata.org/) dataframe and filter by year.

In [129]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_ornaments.titles import set_title_and_subtitle

df = pd.read_csv('https://raw.githubusercontent.com/bianconif/graphic_communication_notebooks/refs/heads/master/data/eu-electricity-production.csv',comment='#')
df = df[df['Year'] == (year := 2023)]
print(df.head(10))

     Country  Year            Source  Value
207  Austria  2023         Bioenergy   4.56
208  Austria  2023              Coal   0.00
209  Austria  2023               Gas   7.50
210  Austria  2023             Hydro  40.67
211  Austria  2023           Nuclear   0.00
212  Austria  2023      Other Fossil   3.32
213  Austria  2023  Other Renewables   0.00
214  Austria  2023             Solar   6.39
215  Austria  2023              Wind   8.34
432  Belgium  2023         Bioenergy   3.80


Add the country codes (ISO 3166)

In [130]:
df_codes = pd.read_csv('https://raw.githubusercontent.com/bianconif/graphic_communication_notebooks/refs/heads/master/data/countries_iso3166b.csv', comment='#')
print(df_codes.head())

HTTPError: HTTP Error 404: Not Found

In [ ]:
df = df.merge(df_codes[['country', 'iso3']], left_on='Country', right_on='country')
print(df.head(10))

Aggregate the primary sources of electricity into the following groups: *fossil*, *nuclear*, *solar*, *wind*, *hydro* and *other renewables*.

In [ ]:
aggregated_classes = {'Bioenergy': 'Other renewables', 'Coal': 'Fossil', 'Gas': 'Fossil', 'Hydro': 'Hydro',
                      'Nuclear': 'Nuclear', 'Other fossil': 'Fossil', 'Other Renewables': 'Other renewables',
                      'Solar': 'Solar', 'Wind': 'Wind'}
df['Source-aggregated'] = df['Source'].map(aggregated_classes)
print(df.head())

Aggregate the electricity production values by country and source.

In [ ]:
df = df.groupby(by=['Country', 'iso3', 'Source-aggregated']).agg({'Value': 'sum'}).reset_index()
print(df.head(10))

Sort the countries by total electricity production in the selected year and retain the first `n_countries`.

In [ ]:
#Sort the countries by total electricity production
df_total_by_country = df.groupby(by='Country').agg('sum').reset_index()
df_total_by_country = df_total_by_country.sort_values(by='Value', ascending=False)

#Retain the first n_countries
top_countries = df_total_by_country.iloc[0:(n_countries := 10)]['Country']
df = df[df['Country'].isin(top_countries)]

print(top_countries)

## Create the stacked bar chart

### Define a colour palette

Define a qualitative colour palette for the electricity production sources. Use shades of red, blue and orange respectively for fossil, hydro, and solar; of green for other renewables. Palette generated with [ColorBrewer 2.0](https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=6).

In [ ]:
palette = {'Fossil': '#e41a1c', 'Hydro': '#377eb8', 'Solar': '#ff7f00', 'Other renewables': '#4daf4a',
           'Nuclear': '#a65628', 'Wind': '#984ea3'}

### Create the chart

Create an empty figure (`fig`) with as many sub-plots (`axes`) as electricity sources using [Matplotlib's object oriented interface](https://matplotlib.org/stable/users/explain/figure/api_interfaces.html) [1]. Arrange the sub-plots into two columns. Iterate through the electricity sources and generate one bar chart for each of them [`matplotlib.axes.Axes.bar`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.bar.html) method. Furthermore:
- Store the overall maximum value of power output by source for uniform scaling of the *y* axis across the sub-plots (`max_val`);
- Use the `hspace` parameter to define the vertical spacing between the sub-plots (values are in fraction of the average sub-plots height)

In [ ]:
max_val = df['Value'].max()
grid_spec = {'hspace': 0.4}

n_cols = 2
n_rows = nrows=df.groupby(by='Source-aggregated').ngroups//n_cols

fig, axes = plt.subplots(n_rows, ncols=2, figsize=(12.8,8), gridspec_kw=grid_spec)
axes = axes.flatten()


for idx, (class_name, class_grp) in enumerate(df.groupby(by='Source-aggregated')):

    #Sort the bars from left to right by total electricity production in descending order
    class_grp = class_grp.set_index(keys='Country')
    class_grp = class_grp.loc[top_countries]
    class_grp = class_grp.reset_index()
    
    values = class_grp['Value'].to_numpy()

    #Create the stack for the current class of electricity source
    axes[idx].bar(x=class_grp['iso3'], height=values, label=class_name, facecolor=palette[class_name])

    #Apply uniform scale to the y axis
    axes[idx].set_ylim([0,max_val])

    #Despine
    axes[idx].spines[['top', 'right', 'left']].set_visible(False)

    #Add name of electricity source
    axes[idx].text(x=1.0, y=1.0, s=class_name, ha='right', va='top', color=palette[class_name],
                   transform=axes[idx].transAxes)
    

## Get the appearance right

### Add a horizontal grid

Use [matplotlib.axes.Axes.grid](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.grid.html) to add a horizontal grid and [Axes.set_axisbelow](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.grid.html) to send the grid to the background (just comment out this call if you'd rather have the grid in the foreground).

In [ ]:
for ax in axes:
    ax.grid(visible=None, which='major', axis='y')
    ax.set_axisbelow(True)
display(fig)

## Add title and subtitle

Add a title and subtitle to the chart with [`mpl_ornaments.set_title_and_subtitle`](https://bianconif.github.io/mpl_ornaments/reference/).

In [ ]:
title = f'Electricity production in the EU by source ({year})'
subtitle = f'Top {n_countries} countries, in TWh/year. Source: EMBER'
set_title_and_subtitle(fig=fig, title=title, subtitle=subtitle, alignment='left', h_offset=90)
display(fig)

## References

[1] Bianconi F. [Data and process visualisation for graphic communication: A hands-on approach with Python](https://link.springer.com/book/10.1007/978-3-031-57051-3). Cham: Springer, 2024.

## Licence

Copyright © 2025 [Francesco Bianconi](https://www.bianconif.net/Home.html). All rights reserved.